# Matrix_Factorization_ALS {-}
#### Team: Ground Truth - Connor DePalma & Daniel Pelis {-} 
#### EE627 {-}

In [ ]:
import numpy as np
import pandas as pd
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [ ]:
train_data = sc.textFile("./data/trainItem.data")
train_ratings = train_data.map(lambda l: l.split(','))\
 .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
train_data = sc.textFile("./data/trainItem.data")
train_ratings = train_data.map(lambda l: l.split(','))\
 .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
rank = 10
numIterations = 10
model = ALS.train(train_ratings, rank, numIterations)

In [ ]:
del train_data
del train_ratings

In [ ]:
testFile = sc.textFile("./data/testItem.data")
test_ratings = testFile.map(lambda l: l.split(','))\
 .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
testdata = test_ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))

In [ ]:
predictions.coalesce(1).saveAsTextFile('./data/preds_2')

#### Rather than creating the dataframe directly, we found more success in saving it as a file first. By doing this we are essentially saving our data in case anything memory related fails, which happened alot on our machines during these tests. If anything fails due to memory or resource constraints after this point, we are able to restart the notebook and are then only required to run in import statements prior to this point, thus saving a lot of resources. {-}  

In [ ]:
# preds = predictions.coalesce(1).toDF().toPandas()

In [ ]:
preds = pd.read_csv('./data/preds_2/part-00000', dtype=str, names=['_1', '_2', '_3'])

In [ ]:
preds['_1'] = preds['_1'].str[1:].astype(np.int64)
preds['_3'] = preds['_3'].str[:-1].astype(np.float64)
preds['_2'] = preds['_2'].astype(np.int64)

In [ ]:
preds

In [ ]:
preds = preds.sort_values(['_1'], ascending=True) \
    .groupby(['_1'], sort=False) \
    .apply(lambda x: x.sort_values(['_3'], ascending=False)) \
    .reset_index(drop=True)

In [ ]:
preds

In [ ]:
# preds.to_csv("./data/preds.csv", index=False)

In [ ]:
dataDir='./data/'
file_name_ratings=dataDir + 'testItem.data'
ratings = open(file_name_ratings, 'r')
#preds = pd.read_csv(dataDir + 'preds.csv')
out_file = dataDir + 'submission_3.csv'

with open(out_file,'w') as f:
    row = 'TrackID,Predictor\n'
    f.write(row)

index = 0
while ratings:
    if index > 119973:
        break
    # grab first six rows in df
    user_preds_user_id = preds.iloc[index,0]
    user_preds = []
    for x in range(6):
        if(preds.iloc[index,0] == user_preds_user_id):
            user_preds.append(preds.iloc[index])
            index += 1
    # user_preds = preds[index:index+6]
    

    # Load user rows into their own data frame with extra col intialized at 0
    user_df = pd.DataFrame(data=user_preds)
    user_df['_4'] = 0
    
    # Sort user df by rank and assign top three with ones
    i = 0
    for i in range(len(user_df['_1'])):
        if i < 3:
            user_df.iloc[i,3] = 1

    user_id = int(user_df.iloc[0,0])

    lines = []
    while len(lines) < 6:
        line = ratings.readline()
        data = line.strip().split(',')
        if str(user_id) == str(data[0]):
            lines.append(data)

    with open(out_file,'a') as f:
        for line in lines:
            userId = line[0]
            item_id = line[1]
            df_row = user_df.loc[user_df['_2'] == int(item_id)]
            try:
                value = df_row['_4'].values[0]
                row = f'{userId}_{item_id},{value}\n'
            except:
                row = f'{userId}_{item_id},0\n'
            f.write(row)